# Mise en place

In [2]:
import pandas as pd
import sqlite3

In [3]:
connection = sqlite3.connect("olist.db")

df = pd.read_sql_query("SELECT * FROM TrainingDataset",connection)

connection.close()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98344 entries, 0 to 98343
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   review_id                      98344 non-null  object 
 1   order_id                       98344 non-null  object 
 2   review_score                   98344 non-null  int64  
 3   review_comment_title           98344 non-null  object 
 4   review_comment_message         98344 non-null  object 
 5   review_creation_date           98344 non-null  object 
 6   review_answer_timestamp        98344 non-null  object 
 7   customer_id                    98344 non-null  object 
 8   order_status                   98344 non-null  object 
 9   order_purchase_timestamp       98344 non-null  object 
 10  order_approved_at              98344 non-null  object 
 11  order_delivered_carrier_date   98344 non-null  object 
 12  order_delivered_customer_date  95542 non-null 

# Model naif

# Model avec les délais de livraison

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98344 entries, 0 to 98343
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   review_id                      98344 non-null  object 
 1   order_id                       98344 non-null  object 
 2   review_score                   98344 non-null  int64  
 3   review_comment_title           98344 non-null  object 
 4   review_comment_message         98344 non-null  object 
 5   review_creation_date           98344 non-null  object 
 6   review_answer_timestamp        98344 non-null  object 
 7   customer_id                    98344 non-null  object 
 8   order_status                   98344 non-null  object 
 9   order_purchase_timestamp       98344 non-null  object 
 10  order_approved_at              98344 non-null  object 
 11  order_delivered_carrier_date   98344 non-null  object 
 12  order_delivered_customer_date  95542 non-null 

In [26]:
print(type(df['score']))
df['score']

<class 'pandas.core.series.Series'>


0        1
1        1
2        0
3        1
4        1
        ..
98339    1
98340    1
98341    1
98342    0
98343    0
Name: score, Length: 98344, dtype: int64

In [27]:
print(type(df[['score']]))
df[['score']]

<class 'pandas.core.frame.DataFrame'>


,score
0,1
1,1
2,0
3,1
4,1
...,...
98339,1
98340,1
98341,1
98342,0


In [16]:
from sklearn.model_selection import train_test_split
y = df['score']
X = df[["produit_recu","temps_livraison","retard_livraison"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [21]:
df.temps_livraison.max()

208.0

In [24]:
from sklearn.impute import SimpleImputer

# Je choisis la stratgéie de remplacer par le maximu
imputer_temps_livraison = SimpleImputer(strategy="constant",fill_value=X_train.temps_livraison.max())

# j'ajuste mon transformeur pour qu'il prenne la valeur du max sur le jeu de train
imputer_temps_livraison.fit(X_train[["temps_livraison"]])

# je remplace les valeurs manquantes du jeu de train par le max calculé (sur le jeu de train)
X_train[["temps_livraison"]] = imputer_temps_livraison.transform(X_train[["temps_livraison"]])

# je remplace les valeurs manquantes du jeu de test par le max calculé (sur le jeu de train)
X_test[["temps_livraison"]] = imputer_temps_livraison.transform(X_test[["temps_livraison"]])

In [30]:
# Je choisis la stratgéie de remplacer par le maximum
imputer_retard_livraison = SimpleImputer(strategy="constant",fill_value=X_train.retard_livraison.max())

# j'ajuste mon transformeur pour qu'il prenne la valeur du max sur le jeu de train
imputer_retard_livraison.fit(X_train[["retard_livraison"]])

# je remplace les valeurs manquantes du jeu de train par le max calculé (sur le jeu de train)
X_train[["retard_livraison"]] = imputer_retard_livraison.transform(X_train[["retard_livraison"]])

# je remplace les valeurs manquantes du jeu de test par le max calculé (sur le jeu de train)
X_test[["retard_livraison"]] = imputer_retard_livraison.transform(X_test[["retard_livraison"]])

In [31]:
X_train["retard_livraison"].isna().value_counts()

retard_livraison
False    78675
Name: count, dtype: int64

In [32]:
X_test["retard_livraison"].isna().value_counts()

retard_livraison
False    19669
Name: count, dtype: int64

In [33]:
X_train.head()

,produit_recu,temps_livraison,retard_livraison
95927,0,25.0,4.0
63215,1,7.0,-15.0
42272,1,3.0,-14.0
90960,1,11.0,-8.0
89768,1,8.0,-17.0


In [34]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

In [35]:
model.fit(X_train,y_train)

LogisticRegression()

In [ ]:
from sklearn.metrics import recall_score, accuracy_score, f1_score

recall = recall_score(y_train, model.predict(X_train))
acc = accuracy_score(y_train, model.predict(X_train))
f1 = f1_score(y_train, model.predict(X_train))

print(f"Pour le jeu d'entrainement: \n le recall est de {recall}, \n l'accuracy de {acc} \n le f1 score de {f1}")

Pour le jeu d'entrainement: 
 le recall est de 0.9852335996126845, 
 l'accuracy de 0.6394661582459485 
 le f1 score de 0.7594249607735041


In [37]:
recall = recall_score(y_test, model.predict(X_test))
acc = accuracy_score(y_test, model.predict(X_test))
f1 = f1_score(y_test, model.predict(X_test))

print(f"Pour le jeu de test: \n le recall est de {recall}, \n l'accuracy de {acc} \n le f1 score de {f1}")

Pour le jeu de test: 
 le recall est de 0.9834573304157549, 
 l'accuracy de 0.6441608622705781 
 le f1 score de 0.7625123002273422


Consignes:
.1  Creer des variables explicatives alternatives (temps_livraison, retard_livraison) en gérant les valeurs extremes par la technique de plafond.
    - Essayer de voir si avec ces variables le modèles s'améliore
.2 Essayer de la lancer un DecisionTreeClassifier (avec les 3 variables précédantes) recherchez la meilleur valeur pour max_depth comme hyperparamètre